## Text Extraction with layout and title

In [13]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
import os 
import pandas as pd
import PyPDF2
import fitz
import pytesseract
import pandas as pd
import os
from pdf2image import convert_from_path
from PIL import Image


In [14]:
def ocr(file_path): #function to extract text using tesseract
    try:
        text = ''

        ## add your own file_location to tesseract.exe
        pytesseract.pytesseract.tesseract_cmd = r'C:/imls-research-grant/pr_files/tesseract/tesseract.exe'


        ## add your own file_location to poppler
        images = convert_from_path(pdf_path=file_path+'.pdf',poppler_path = 'C:\imls-research-grant\pr_files\poppler')
        for count, img in enumerate(images):
            extracted_text = pytesseract.image_to_string(img)
            text += extracted_text
            
            # adding this text for line break  
            text += '****************************************************************************************************'
        
        
        return text
    except:
        return 0
            

In [15]:
def extract_text_from_pdf(file_path): # function to extract text using pdfminer
    try:
        text = ''
        title = ''
        layout = ''
        for page_layout in extract_pages(file_path+'.pdf'):
            for element in page_layout:

                if isinstance(element, LTTextContainer):
                    for text_line in element:
                        try:
                            text += text_line.get_text()
                        except:
                            continue
                        
            if len(text) != 0:  
                
                # adding this text for line break                 
                text += '****************************************************************************************************'
        
        with open('./'+i+'.pdf', 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            try:
                title = reader.metadata['/Title']
            except:
                title = 'NA'
        with fitz.open('./'+i+'.pdf') as doc:
            layout = str(doc[0].rect.width)+' * '+str(doc[0].rect.height)
        
        
        
        
        return text, title, layout
    
    
    except:
        return  0, title, layout



In [16]:
# In the below code we just iterating through each pdf available in directory 
# first we try to exract the text using the pdfminer
# If pdfminer not works then we extract the text using tesseract
# finnally we storing it to dataframe




# give file location to both positive and negative data one after another
p = os.listdir('C:\imls-research-grant\pr_files/2017\data/full_pdf_data/1.Texas_Pub_Candidates')
c = list()
num = 20
count = 0
df = pd.DataFrame(columns = ['pdf_name', 'text', 'title', 'layout'])
for k, i in enumerate(p[:]):
    os.chdir('C:\imls-research-grant\pr_files/2017\data/full_pdf_data/1.Texas_Pub_Candidates/'+i)
    text, title, layout =  extract_text_from_pdf(i)
    if text == 0 or text == '':
        text =  ocr(i)
        if text == 0 or text == '':
            continue
    # print(i, text, title, layout)
    df.loc[len(df.index)] = [i, text, title, layout]


24LOI5BAD2PKTQ6SUMPFNGPTATFJCMBL lt takes more than a kiss.

P+ walelelTinials)

Build your child's health

 

ProtectTexas” Call 1-800-252-9152 or visit www.lmmunizeTexas.com
**************************************************************************************************** Vaccine Billboard Ad 792.0 * 360.0
252QHHKHPARPIPSWWHPFC5MTWPS5TF42 Acknowledgements
This report was researched and written by the staff at the Travis County Health and 
Human  Services  &  Veterans  Service,  Research  &  Planning  Division. Lead  writers 
were  Rachel  Coff,  Korey  Darling,  and  Veronica  Neville,  with  the  support  of  other
Division  staff,  Blanca  Tapia  Leahy  (Division  Director),  and  Sherri  E.  Fleming 
(Executive Manager).
Questions or Comments?
For  questions,  more  information,  or  to  request  a  presentation,  please  contact  the 
Research & Planning Division at R&P_HHS@co.travis.tx.us.
***************************************************************************************

## Structural_features

In [17]:
title = list()
total_chracter_l = list()
total_words_l = list()
total_lines_l = list()
avg_words_page_l = list()
avg_lines_page_l = list()
space_char_per_l = list()
avg_alphanum_per_l = list()
line_ratio_l = list()
total_upcase_page_l = list()
total_alphnum_line_l = list()

In [18]:

#extracting structural faetures based above extracted text
for text in df['text']:
    page_count = 0
    total_lines = 0
    total_words = 0
    total_chracter = 0
    total_spaces = 0
    total_alphnum_page = 0
    total_upcase_page = 0
    total_alphnum_line = 0
    max = None
    min = None
    for page in text.split('****************************************************************************************************'):
        num_lines_page = 0
        num_words_page = 0
        num_char_page = 0
        num_spaces_page = 0
        num_alphnum_page = 0
        num_upcase_page = 0
        num_alphnum_line = 0
        if len(page) != 0 :
            for line in page.split('\n'):
                if len(line) != 0 :
                    #print(len(line.split(" ")))
                    m = line.split(" ")
                    if(line[0].isupper()):
                        num_upcase_page += 1
                        
                    num_spaces_page += len(m)-1
                    num_char_page += len(line)
                    num_words_page += len(m)
                    num_lines_page += 1
                    
                    if max==None and min == None:
                        max = len(line)
                        min = len(line)
                    if len(line) > max:
                        max = len(line)
                    if len(line) < min:
                        min = len(line)
                        
                    for i in m:
                        if(i.isalpha() or i.isnumeric()):
                            continue
                        else:
                            if(i.isalnum()):
                                num_alphnum_page += 1
                                
                    if(line[0].isalpha() or line[0].isnumeric()):
                            continue
                    else:
                        if(line[0].isalnum()):
                            num_alphnum_line += 1
            page_count += 1
                                
            total_alphnum_line += num_alphnum_line
            total_upcase_page += num_upcase_page
            total_alphnum_page += num_alphnum_page
            total_lines += num_lines_page
            total_words += num_words_page
            total_chracter += num_char_page
            total_spaces += num_spaces_page
    try:
        avg_lines_page = total_lines/page_count
        avg_words_page = total_words/page_count
        space_char_per = round((total_spaces / total_chracter) * 100, 2)
        avg_alphanum_per = (total_alphnum_page / total_words) * 100
        line_ratio  = min/max
    except:
        total_chracter_l.append(0)
        total_words_l.append(0)
        total_lines_l.append(0)
        avg_words_page_l.append(0)
        avg_lines_page_l.append(0)
        space_char_per_l.append(0)
        avg_alphanum_per_l.append(0)
        line_ratio_l.append(0)
        total_upcase_page_l.append(0)
        total_alphnum_line_l.append(0)
    total_chracter_l.append(total_chracter)
    total_words_l.append(total_words)
    total_lines_l.append(total_lines)
    avg_words_page_l.append(avg_words_page)
    avg_lines_page_l.append(avg_lines_page)
    space_char_per_l.append(space_char_per)
    avg_alphanum_per_l.append(avg_alphanum_per)
    line_ratio_l.append(line_ratio)
    total_upcase_page_l.append(total_upcase_page)
    total_alphnum_line_l.append(total_alphnum_line)
     
df["total_chracter"] = total_chracter_l
df["total_words"] = total_words_l
df["total_lines"] = total_lines_l
df["avg_words_page"] = avg_words_page_l
df["avg_lines_page"] = avg_lines_page_l
df["space_char_per"] = space_char_per_l
df["avg_alphanum_per"] = avg_alphanum_per_l
df["line_ratio"] = line_ratio_l
df["total_upcase_page"] = total_upcase_page_l
df["total_alphnum_line"] = total_alphnum_line_l



                           pdf_name  \
0  24LOI5BAD2PKTQ6SUMPFNGPTATFJCMBL   
1  252QHHKHPARPIPSWWHPFC5MTWPS5TF42   

                                                text  \
0  lt takes more than a kiss.\n\nP+ walelelTinial...   
1  Acknowledgements\nThis report was researched a...   

                                             title         layout  
0                             Vaccine Billboard Ad  792.0 * 360.0  
1  Microsoft PowerPoint - ACS Chartbook 2006_FINAL  612.0 * 792.0   2 2 2 2


In [19]:
df.to_csv('s_p.csv') 